In [34]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [12]:
file_url = 'data/data.csv'
dataframe = pd.read_csv(file_url)
print(dataframe.shape)
dataframe.head(20)

(500, 10)


,Asset ID,Asset Type,Floor,Room,Installation Date,Manufacturer,Operational Time (hrs),Work Orders,Repairs,Last Serviced Date
0,1,Elevator,7,103,1/6/2020,Manufacturer_4,39313,0,1,6/19/2023
1,2,Plumbing System,6,109,6/11/2022,Manufacturer_2,20012,0,1,1/6/2023
2,3,Fire Alarm,3,104,7/24/2019,Manufacturer_4,21546,10,5,9/24/2022
3,4,Elevator,1,105,1/4/2021,Manufacturer_4,47038,1,1,5/23/2023
4,5,Plumbing System,2,104,11/28/2022,Manufacturer_1,20022,10,3,1/29/2023
5,6,Fire Alarm,2,103,7/29/2021,Manufacturer_3,1602,9,4,10/3/2022
6,7,Fire Alarm,6,108,8/23/2020,Manufacturer_2,15650,2,3,1/13/2023
7,8,HVAC,2,106,6/3/2021,Manufacturer_2,33932,1,4,8/9/2023
8,9,Fire Alarm,6,108,7/30/2023,Manufacturer_5,24332,5,4,2/13/2023
9,10,Elevator,7,101,7/24/2019,Manufacturer_4,18722,10,2,5/29/2023


In [36]:
reference_date = pd.Timestamp.now()

dataframe['Installation Date'] = pd.to_datetime(dataframe['Installation Date'], format='%m/%d/%Y')
dataframe['Days Since Installation'] = abs((dataframe['Installation Date'] - reference_date).dt.days)

dataframe['Last Serviced Date'] = pd.to_datetime(dataframe['Last Serviced Date'], format='%m/%d/%Y')
dataframe['Days Since Last Serviced'] = abs((dataframe['Last Serviced Date'] - reference_date).dt.days)

dataframe.head(20)


,Asset ID,Asset Type,Floor,Room,Installation Date,Manufacturer,Operational Time (hrs),Work Orders,Repairs,Last Serviced Date,Days Since Installation,Days Since Last Serviced
0,1,Elevator,7,103,2020-01-06,Manufacturer_4,39313,0,1,2023-06-19,1350,90
1,2,Plumbing System,6,109,2022-06-11,Manufacturer_2,20012,0,1,2023-01-06,463,254
2,3,Fire Alarm,3,104,2019-07-24,Manufacturer_4,21546,10,5,2022-09-24,1516,358
3,4,Elevator,1,105,2021-01-04,Manufacturer_4,47038,1,1,2023-05-23,986,117
4,5,Plumbing System,2,104,2022-11-28,Manufacturer_1,20022,10,3,2023-01-29,293,231
5,6,Fire Alarm,2,103,2021-07-29,Manufacturer_3,1602,9,4,2022-10-03,780,349
6,7,Fire Alarm,6,108,2020-08-23,Manufacturer_2,15650,2,3,2023-01-13,1120,247
7,8,HVAC,2,106,2021-06-03,Manufacturer_2,33932,1,4,2023-08-09,836,39
8,9,Fire Alarm,6,108,2023-07-30,Manufacturer_5,24332,5,4,2023-02-13,49,216
9,10,Elevator,7,101,2019-07-24,Manufacturer_4,18722,10,2,2023-05-29,1516,111


In [13]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 400 samples for training and 100 for validation


In [24]:
train_dataframe.head()

,Asset ID,Asset Type,Floor,Room,Installation Date,Manufacturer,Operational Time (hrs),Work Orders,Repairs,Last Serviced Date,Days Since Last Serviced
0,1,Elevator,7,103,1/6/2020,Manufacturer_4,39313,0,1,2023-06-19,90
1,2,Plumbing System,6,109,6/11/2022,Manufacturer_2,20012,0,1,2023-01-06,254
3,4,Elevator,1,105,1/4/2021,Manufacturer_4,47038,1,1,2023-05-23,117
4,5,Plumbing System,2,104,11/28/2022,Manufacturer_1,20022,10,3,2023-01-29,231
5,6,Fire Alarm,2,103,7/29/2021,Manufacturer_3,1602,9,4,2022-10-03,349


In [7]:
# def dataframe_to_dataset(dataframe):
#     dataframe = dataframe.copy()
#     labels = dataframe.pop("target")
#     ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
#     ds = ds.shuffle(buffer_size=len(dataframe))
#     return ds


# train_ds = dataframe_to_dataset(train_dataframe)
# val_ds = dataframe_to_dataset(val_dataframe)

NameError: name 'labels' is not defined